In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import statsmodels.api as sm
import statsmodels.formula.api as smf

from itertools import product
from ukbb_recessive.regression.regressions import run_regressions, save_table_for_paper, get_formula, get_target_family

# Load regressions datasets

First, we load datasets for the regression analysis saved on the previous step using script `0_create_dataset.ipynb`. 

In [ ]:
output_path = "../../../../data/tables"
datasets_path = '.../450k/datasets'

datasets_dict = {
    # 'Cassa': pd.read_csv(f"{datasets_path}/Cassa.csv", sep='\t'),
    # 'pLI': pd.read_csv(f"{datasets_path}/PLI.csv", sep='\t'),
    # 'Weghorn': pd.read_csv(f"{datasets_path}/Weghorn-drift.csv", sep='\t'),
    'Roulette': pd.read_csv(f"{datasets_path}/Roulette.csv", sep='\t'),
}

# samples of interest: European & non-related without hom and comp_het
european_non_rel_samples = (
    ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.no_hom_comp_het.txt"
)

with open(european_non_rel_samples, 'r') as f:
    european_non_rel_samples = [l.strip() for l in f.readlines()]

print (f"Number of european non-related samples without hom and comp_het: {len(european_non_rel_samples)}\n")

for dataset_name, dataset in datasets_dict.items():
    print (f"Number of samples in {dataset_name}: {datasets_dict[dataset_name].shape[0]}")
    # dataset = dataset[dataset['eid'].astype(str).isin(european_non_rel_samples)].copy()

    # dataset['multiple_deprivation'] = (dataset['multiple_deprivation_engand']
    #                                    .combine_first(dataset['multiple_deprivation_wales'])
    #                                    .combine_first(dataset['multiple_deprivation_scotland']))
    
    # datasets_dict[dataset_name] = dataset
    
    datasets_dict[dataset_name] = dataset[dataset['eid'].astype(str).isin(european_non_rel_samples)].copy()

    print (f"Number of samples in {dataset_name} after filtration: {datasets_dict[dataset_name].shape[0]}")

Here are all s_het-based genetic burdens, calculated based on different gene sets and variants (PLPs or singleton LOFs):

In [ ]:
for col in datasets_dict[list(datasets_dict.keys())[0]].columns:
    if col[:5] == 's_het':
        print (col)

In [ ]:
dataset.columns.tolist()[-20:]

# Define analyses

In [ ]:
covariates = ['multiple_deprivation_engand', 'edu_deprivation_england',
              'housing_deprivation_england', 'income_deprivation_england','health_deprivation_england']

# we looked into these phenotypes as covariates
covariate_deprivation_regressions = {
    's_hets': ['s_het_recessive_all', 's_het_lof_without_AR'] + list(product(['s_het_recessive_all', 's_het_lof_without_AR'], covariates)),
    'targets': ['childlessness', 'years_of_edu', 'diagnosis_total_ICD10_cnt_log', 'is_blond'],
    'families': ['binomial', 'gaussian', 'gaussian', 'binomial'],
    'genders': ['all'], 
    'n_tests_correction': 10,
    'filter_dataset': lambda dataset: dataset[~dataset['multiple_deprivation_engand'].isnull()]
}
# we looked into id vs rest
id_vs_rest_covariate_deprivation_regressions = {
    's_hets': ['s_het_recessive_AR_without_ID', 's_het_recessive_ID_total'] + list(product(['s_het_recessive_AR_without_ID', 's_het_recessive_ID_total'], covariates)),
    'targets': ['childlessness', 'years_of_edu', 'diagnosis_total_ICD10_cnt_log', 'is_blond'],
    'families': ['binomial', 'gaussian', 'gaussian', 'binomial'],
    'genders': ['all'], 
    'n_tests_correction': 14,
    'filter_dataset': lambda dataset: dataset[~dataset['multiple_deprivation_engand'].isnull()]
}

analyses = {
    # 'covariate_deprivation': covariate_deprivation_regressions,
    'id_vs_rest_covariate_deprivation': id_vs_rest_covariate_deprivation_regressions
}

# Regressions

In [ ]:
for analysis, analysis_cfg in analyses.items():

    all_results = defaultdict(list)

    print (f"Running {analysis} analysis\n")

    # read analysis config
    s_hets = analysis_cfg['s_hets']
    targets = analysis_cfg['targets']
    families = analysis_cfg['families']
    genders = analysis_cfg['genders']   
    filter_dataset_func = analysis_cfg['filter_dataset']
    n_tests_correction = analysis_cfg['n_tests_correction']
     
    
    # iterate over different s-het datasets
    for dataset_key in datasets_dict:
        print ("\tProcessing", dataset_key, flush=True)

        # iterate over different s-het burdens
        for s_het in s_hets:
            if not (isinstance(s_het, list) or isinstance(s_het, tuple)) :
                s_het = [s_het]

            analysis_tag = f'{analysis} regressions on {str(s_het)}'.replace("'", '')

            print (f"\t\tRun {analysis_tag}")

            dataset = filter_dataset_func(datasets_dict[dataset_key]).copy()
            print (f"\t\t\tDataset size after filtration: {dataset.shape}")

            regression_result = run_regressions(dataset=dataset, 
                                                targets=targets, 
                                                families=families, 
                                                analysis_tag=analysis_tag, 
                                                genders=genders, 
                                                s_het_list=s_het, 
                                                tab_offset='\t\t\t', 
                                                n_tests_correction=n_tests_correction)
            
            all_results[dataset_key].append(regression_result)

            print()

    all_results = {dataset_key: pd.concat(all_results[dataset_key]) for dataset_key in all_results}

    save_table_for_paper(all_results, f"{output_path}/table_{analysis}_analysis.xlsx")

    print()

In [ ]:
! ls -lah ../../../../data/tables